In [80]:
from random import shuffle, randint
from tqdm import tqdm_notebook
import pandas as pd

In [44]:
with open("data/2.txt", encoding="cp1251") as f:
    txt = f.read()
    
import re
from collections import Counter

BOS = "BOS"
EOS = "EOS"
UNK = "UNK"

def prepare_sentences(txt, word_threshold=2, stage_train=True):
    
    # заменяем многие пробелы на один
    txt = re.sub("\s+", " ", txt) \
        .lower() # и переводим в нижний регистр
    
    # заменяем символы переноса строки на пробелы (абзацы нам не нужны)
    text = txt.replace("\n", " ")
    # разбиваем текст на предложения по знакам препинания
    sentences = re.split("[!\?\.]+", txt.replace("\n", " "))
    
    # оставляем только alphanumeric
    # W — любой символ, кроме буквенного или цифрового символа или знака подчёркивания
    clean_sentences = [re.split("\W+", s) for s in sentences]
    
    # заменяем числа на NUM
    clean_sentences = [[w.replace("\d+", "NUM") for w in s if w] for s in clean_sentences]
    
    # вводим тег UNKNOWN: UNK
    if stage_train:

        counter = Counter()

        for s in clean_sentences:
            for w in s:
                counter[w] += 1
    
        print("Filtered out word types :", len([w for w in counter if counter[w] <= word_threshold]))
        print("Filtered out words count:", sum([counter[w] for w in counter if counter[w] <= word_threshold]))
    
        # выкидываем редкие, и заменяем их на специальный тег
        clean_sentences = [[w if counter[w] > word_threshold else UNK for w in s] for s in clean_sentences]            
    
    word2index = { BOS: 0, EOS: 1, UNK: 2}
    index2word = { 0: BOS, 1: EOS, 2: UNK}
    
    counter = max(word2index.values()) + 1

    for s in clean_sentences:
        for w in s:
            if not w in word2index:
                word2index[w] = counter
                index2word[counter] = w
                counter += 1
                
    return word2index, index2word, clean_sentences

In [45]:
word2index, index2word, clean_sentences = prepare_sentences(txt)

print("Total number of sentences :\t", len(clean_sentences))
print("Total number of words     :\t", sum([len(sent) for sent in clean_sentences]))
print("Total number of word types:\t", len(set([w for sent in clean_sentences for w in sent])))

Filtered out word types : 25498
Filtered out words count: 31780
Total number of sentences :	 16662
Total number of words     :	 246974
Total number of word types:	 10117


In [46]:
def augment(sentence, context_size):
    """
        Добиваем символы начала и конца строки к каждому предложению
    """
    return [BOS] * context_size + sentence + [EOS] * context_size

def enumerate_sentences(clean_sentences, context_size, word2index):
    """
        Добиваем символами начала и конца и конвертируем слова в индексы
    """

    contexts = []
    targets = []
    UNK_id = word2index[UNK]

    for sentence in clean_sentences:

        aligned_sentence =  augment(sentence, context_size) 

        for i in range(context_size, len(sentence) - context_size, 1):
            
            # берём предшествующий контекст
            context = aligned_sentence[i - context_size:i]
            context = [word2index[c] if c in word2index else UNK_id for c in context]
            target = word2index[aligned_sentence[i]] if aligned_sentence[i] in word2index else UNK_id
            
            contexts.append(context)
            targets.append(target)
    
    return contexts, targets

In [65]:
txt = re.sub("\s+", " ", txt) \
    .lower() # и переводим в нижний регистр

# заменяем символы переноса строки на пробелы (абзацы нам не нужны)
text = txt.replace("\n", " ")
# разбиваем текст на предложения по знакам препинания
sentences = re.split("[!\?\.]+", txt.replace("\n", " "))

# оставляем только alphanumeric
# W — любой символ, кроме буквенного или цифрового символа или знака подчёркивания
clean_sentences = [re.sub("\W+", " ", s).strip() for s in sentences]

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

lemm_sents = []
for s in tqdm_notebook(clean_sentences):
    new_sent = []
    for w in s.split():
        lemm = morph.parse(w)[0].normal_form
        new_sent.append(lemm)
    lemm_sents.append(" ".join(new_sent))

In [66]:
print(lemm_sents[100])

я он два раз как ты видеть


In [67]:
shuffled_sents = []

for s in lemm_sents:
    s = s.split()
    shuffle(s)
    shuffled_sents.append(" ".join(s))

In [69]:
shuffled_sents[0], lemm_sents[0]

('тот армия 1811 начаться точно россия и который 1812 1811 кормить го сила год стягиваться двинуться этот год конец западный с европа год россия го к человек так восток в сила вооружение миллион и который считать сила к сосредоточение же с перевозить усиленный запад на и с граница',
 'с конец 1811 го год начаться усиленный вооружение и сосредоточение сила западный европа и в 1812 год сила этот миллион человек считать тот который перевозить и кормить армия двинуться с запад на восток к граница россия к который точно так же с 1811 го год стягиваться сила россия')

In [126]:
train = lemm_sents[:round(len(lemm_sents)*0.9)]
test = list(zip(lemm_sents, shuffled_sents))[round(len(lemm_sents)*0.9):]

In [127]:
len(train), len(test)

(14996, 1666)

In [128]:
df_train = pd.Series(train, name="true").to_frame()
df_train.head()

,true
0,с конец 1811 го год начаться усиленный вооруже...
1,12 июнь сила западный европа переслать граница...
2,миллион человек совершать друг против друг так...
3,что произвести это необычайный событие
4,какой быть причина он


In [138]:
df_train.to_csv("data/train.tsv", sep="\t")

In [129]:
labels = []
for true, false in test:
    if randint(0, 1):
        labels.append((true, false, 1))
    else:
        labels.append((false, true, 0))

In [136]:
df_test = pd.DataFrame().from_records(labels, columns=["text1", "text2", "label"])
df_test.head()

,text1,text2,label
0,что есть власть,что власть есть,1
1,власть есть совокупность воля перенести на оди...,лицо воля один перенести есть власть совокупно...,1
2,какой переноситься при на воля условие лицо ма...,при какой условие переноситься воля масса на о...,0
3,при условие выражение лицо воля весь человек,лицо воля выражение человек весь условие при,1
4,власть то есть есть власть,то есть власть есть власть,0


In [139]:
df_test.drop("label", 1).to_csv("data/task.tsv", sep="\t")

In [141]:
df_test["label"].to_csv("data/true_labels.tsv", sep="\t", index=False)